In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F

from torch.utils.data import DataLoader
import numpy as np
import sys
sys.path += ['..', '../..', '../../..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime




%load_ext autoreload
%autoreload 2


In [2]:
from data_preparation import (
    Session, match_time, data_prep, sample_prep, stack_samples
)
sessions_v0 = Session.load_multiple_session('../../data/v0',         
    camera_log_name = 'PicameraV2', 
    angular_speed_control_log_name='AngularSpeedControlV2'
)
sessions_v30Jun = Session.load_multiple_session('../../data/v30Jun')

2024-07-14 16:42:14.421351: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 16:42:15.034875: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


FrameMLToAngularVelocity not found


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x61f5b6f28a00] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x61f5b3910580] moov atom not found


FrameMLToAngularVelocity not found
FrameMLToAngularVelocity not found
FrameMLToAngularVelocity not found


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x61f5b343ed80] moov atom not found


In [3]:
for s in tqdm(sessions_v30Jun+sessions_v0): data_prep(s), sample_prep(s)


100%|██████████| 4/4 [00:08<00:00,  2.23s/it]


In [4]:
x, y = stack_samples(sessions_v30Jun+sessions_v0)
x = np.swapaxes(x, 1, -1)

In [5]:
from main import Model, to_dataset, UpdatingPlotlyLines, to_float32, fit


In [17]:
model = Model().to("cuda")
loader = DataLoader(to_dataset(x, y), batch_size=64)
optimiser = optim.Adam(model.parameters(), lr=1e-4)



In [18]:
from torch.nn import functional as F

fit(
    model, 
    loader, loader,
    optimiser,
    F.huber_loss, 
    100
)

FigureWidget({
    'data': [{'name': 'train_loss', 'type': 'scatter', 'uid': 'e50d7190-147f-40be-ab31-5ceeb291e732', 'x': [], 'y': []},
             {'name': 'val_loss', 'type': 'scatter', 'uid': '597b7480-f82c-4c91-9be8-1c8b69a5bbf8', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

In [ ]:

fig = UpdatingPlotlyLines('epoch', ['loss'])
fig.display()
model.set_learning_rate(3e-4)
loader = DataLoader(to_dataset(x, y), batch_size=64)
for i in range(70):
    model.train_step(loader,next(model.parameters()).device)
    loss = model.eval_step(loader, next(model.parameters()).device)
    fig.append(epoch=i, loss=loss)



FigureWidget({
    'data': [{'name': 'loss', 'type': 'scatter', 'uid': '88c750c4-17b8-4b17-8c7a-7dff575f897b', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

KeyboardInterrupt: 

In [24]:
import onnxscript
print(onnxscript.__version__)

from onnxscript import opset18  # opset 18 is the latest (and only) supported version for now

0.1.0.dev20240714


In [25]:
import onnxruntime
print(onnxruntime.__version__)

ModuleNotFoundError: No module named 'onnxruntime'

In [26]:
onnx_program = torch.onnx.dynamo_export(model, torch.randn(1, 3, 114, 64).to('cuda'))

/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/venv/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:136: UserWarning:

torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.



In [ ]:
import onnxruntime
import onnx

def save_as_onnx(model, sample_x, file='temp.onnx'):
    onnx_program = torch.onnx.dynamo_export(model, sample_x)
    onnx_program.save(file)

    onnx_model = onnx.load(file)
    onnx.checker.check_model(onnx_model)    


class RunONNX: 
    def __init__(self, file):
        self.ort_session = onnxruntime.InferenceSession(file, providers=['CPUExecutionProvider'])

        self.inputs = ort_session.get_inputs()
        self.input_names = [i.name for i in self.inputs]

    def __call__(self, *x):
        return ort_session.run(None, dict(zip(self.input_names, x)))




In [27]:
onnx_program.save("my_onnx_test.onnx")

In [29]:
import onnx
onnx_model = onnx.load("my_onnx_test.onnx")
onnx.checker.check_model(onnx_model)

In [32]:
onnx.checker.check_model(onnx_model)

In [33]:
import onnxruntime


In [34]:
ort_session = onnxruntime.InferenceSession("./my_onnx_test.onnx", providers=['CPUExecutionProvider'])


In [42]:
temp = ort_session.get_inputs()[0]

In [44]:
temp

In [51]:
np.random.randn(1, 3, 114, 64)

array([[[[-0.34891318,  1.0622745 , -1.0824354 , ..., -0.62308735,
           0.30308056,  0.73151482],
         [ 1.47593836,  0.1593238 , -1.0851091 , ...,  0.28027062,
           1.58703302, -1.14244374],
         [ 0.66858544, -0.5142003 , -0.46254399, ...,  0.02913761,
           0.08927546, -0.58115932],
         ...,
         [ 1.12016729, -0.17545865, -0.61865008, ..., -0.578219  ,
          -0.95793702, -0.36375894],
         [-0.38097383, -1.32495184,  1.73576544, ..., -0.58445685,
          -0.43237066,  0.82379146],
         [-0.59438925, -1.18727015, -0.1602842 , ..., -1.08247792,
           1.48551394, -1.00472339]],

        [[-2.49436892, -0.12445479, -0.65731021, ..., -0.11370694,
          -1.51582082,  0.47239198],
         [ 0.46846591,  0.16113942,  0.48170188, ..., -0.99676612,
          -0.16683899,  0.48846919],
         [ 0.84406245, -1.23583475,  0.50303888, ..., -0.24906324,
          -0.68066226, -1.95913042],
         ...,
         [-3.07466088,  0.70778735

In [56]:
for i in range(1000): 
    ort_session.run(None, {temp.name:np.random.randn(1, 3, 114, 64).astype(np.float32)})

In [66]:
x = np.random.randn(1, 3, 114, 64).astype(np.float32)
for i in range(10000): 
    ort_session.run(None, {temp.name:x})

In [64]:
model_cpu = model.cpu()
x = torch.randn(1, 3, 114, 64)

In [65]:
for i in range(10000): 
    model_cpu(x)

In [56]:
for _x, _y in loader:
    _x, _y
    break


px.line(_y)

In [61]:
px.line(model(_x.to('cuda')).detach().cpu())

In [42]:
px.line(y[:512])

In [40]:
y_pred = model(torch.from_numpy(x[:512]).to('cuda'))

In [41]:
px.line(y_pred.cpu().detach().numpy())

In [107]:
for x, y in (dl):
    x, y
x

tensor([[[[168, 167, 167,  ..., 131, 136, 134],
          [168, 167, 167,  ..., 138, 140, 139],
          [168, 167, 167,  ..., 143, 146, 145],
          ...,
          [105, 105, 105,  ..., 104, 100, 112],
          [100, 101, 101,  ...,  95,  96, 105],
          [100, 101, 101,  ...,  95,  96, 105]],

         [[149, 148, 149,  ..., 147, 153, 151],
          [149, 148, 149,  ..., 154, 157, 156],
          [149, 148, 148,  ..., 156, 159, 158],
          ...,
          [106, 106, 106,  ..., 139, 134, 146],
          [100, 101, 104,  ..., 130, 132, 141],
          [100, 101, 104,  ..., 130, 132, 141]],

         [[102, 101, 100,  ..., 140, 144, 142],
          [102, 101, 100,  ..., 147, 148, 147],
          [104, 103, 101,  ..., 150, 153, 152],
          ...,
          [ 88,  88,  88,  ..., 143, 140, 152],
          [ 78,  79,  81,  ..., 134, 133, 142],
          [ 78,  79,  81,  ..., 134, 133, 142]]],


        [[[166, 166, 165,  ..., 122, 118, 104],
          [166, 166, 165,  ..., 123

In [34]:
from torch.nn import functional as F

F.huber_loss(y_pred, to_float32(y[:32]).to('cuda'))

tensor(0.0118, device='cuda:0', grad_fn=<HuberLossBackward0>)

In [32]:
y_pred

tensor([[ 0.0112, -0.0090,  0.0160, -0.0076],
        [ 0.0113, -0.0098,  0.0164, -0.0077],
        [ 0.0111, -0.0096,  0.0165, -0.0072],
        [ 0.0108, -0.0095,  0.0166, -0.0073],
        [ 0.0108, -0.0096,  0.0170, -0.0069],
        [ 0.0107, -0.0098,  0.0176, -0.0067],
        [ 0.0104, -0.0095,  0.0180, -0.0062],
        [ 0.0099, -0.0096,  0.0179, -0.0061],
        [ 0.0102, -0.0091,  0.0185, -0.0057],
        [ 0.0104, -0.0092,  0.0189, -0.0055],
        [ 0.0099, -0.0094,  0.0186, -0.0056],
        [ 0.0098, -0.0099,  0.0181, -0.0057],
        [ 0.0099, -0.0098,  0.0185, -0.0058],
        [ 0.0099, -0.0094,  0.0172, -0.0065],
        [ 0.0099, -0.0098,  0.0157, -0.0074],
        [ 0.0112, -0.0093,  0.0132, -0.0100],
        [ 0.0108, -0.0087,  0.0107, -0.0112],
        [ 0.0101, -0.0092,  0.0086, -0.0121],
        [ 0.0101, -0.0101,  0.0089, -0.0119],
        [ 0.0100, -0.0112,  0.0102, -0.0114],
        [ 0.0100, -0.0121,  0.0115, -0.0111],
        [ 0.0108, -0.0129,  0.0135

In [100]:
DataLoader(ds)

In [87]:
from torch.utils.data import DataLoader, Dataset
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(3, 16, (5, 5), stride=2), 
            nn.ReLU(), 
            nn.Conv2d(16, 32, (5, 5), stride=2), 
            nn.ReLU(), 
            nn.Conv2d(32, 64, (5, 5), stride=2), 
            nn.ReLU(), 
            nn.Flatten(),
            nn.Dropout(0.2), 
        )
        self.denses = nn.Sequential(
            nn.Linear(3520, 64), 
            nn.ReLU(),
            nn.Linear(64, 64), 
            nn.ReLU(),
            nn.Linear(64, 4), 

        )
    def forward(self, x):
        return self.denses(self.conv_net(x/255))
model = Model().to("cuda")

In [81]:

def to_dataset(x, y, x_transform=torch.from_numpy, y_transform = torch.from_numpy):
    assert len(x) == len(y)
    class ds(Dataset):
        def __init__(self):
            pass
        def __len__(self):
            return len(x)

        def __getitem__(self, idx):

            return x_transform(x[idx]), y_transform(y[idx])

    return ds

In [110]:
x.get_device?

Docstring:
get_device() -> Device ordinal (Integer)

For CUDA tensors, this function returns the device ordinal of the GPU on which the tensor resides.
For CPU tensors, this function returns `-1`.

Example::

    >>> x = torch.randn(3, 4, 5, device='cuda:0')
    >>> x.get_device()
    0
    >>> x.cpu().get_device()
    -1
Type:      builtin_function_or_method

In [109]:
x.to?

Docstring:
to(*args, **kwargs) -> Tensor

Performs Tensor dtype and/or device conversion. A :class:`torch.dtype` and :class:`torch.device` are
inferred from the arguments of ``self.to(*args, **kwargs)``.

.. note::

    If the ``self`` Tensor already
    has the correct :class:`torch.dtype` and :class:`torch.device`, then ``self`` is returned.
    Otherwise, the returned tensor is a copy of ``self`` with the desired
    :class:`torch.dtype` and :class:`torch.device`.

Here are the ways to call ``to``:

.. method:: to(dtype, non_blocking=False, copy=False, memory_format=torch.preserve_format) -> Tensor
   :noindex:

    Returns a Tensor with the specified :attr:`dtype`

    Args:
        memory_format (:class:`torch.memory_format`, optional): the desired memory format of
        returned Tensor. Default: ``torch.preserve_format``.

.. method:: to(device=None, dtype=None, non_blocking=False, copy=False, memory_format=torch.preserve_format) -> Tensor
   :noindex:

    Returns a Tensor wit

In [88]:
ds = to_dataset(x, y)

In [89]:
ds

__main__.to_dataset.<locals>.ds

In [78]:
#_x = torch.from_numpy(_x).to('cuda')

y = model(_x)

In [80]:
y.shape

torch.Size([32, 4])

In [65]:
y

tensor([[-0.0782,  0.0688,  0.0153,  0.1222],
        [-0.0779,  0.0692,  0.0126,  0.1219],
        [-0.0788,  0.0675,  0.0159,  0.1222],
        [-0.0794,  0.0670,  0.0139,  0.1226],
        [-0.0762,  0.0708,  0.0149,  0.1219],
        [-0.0777,  0.0689,  0.0161,  0.1211]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [30]:
y

tensor([[0.0096, 0.0000, 0.0000,  ..., 0.1325, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0336,  ..., 0.0000, 0.0000, 0.1819]],
       device='cuda:0', grad_fn=<NativeDropoutBackward0>)

In [25]:
nn.Conv2d(16, 33, 3, stride=2)(torch.randn(20, 16, 50, 100))

tensor([[[[ 5.0888e-01, -4.8398e-01, -7.2214e-01,  ...,  2.2755e-01,
           -3.1244e-01, -2.0398e-01],
          [-2.1482e-02, -7.5273e-01,  4.4771e-01,  ..., -1.1518e+00,
           -4.7848e-02, -4.2614e-01],
          [ 3.5180e-02,  1.8799e-02,  1.2645e-01,  ..., -3.9079e-01,
            1.1521e+00, -4.9467e-01],
          ...,
          [ 6.4461e-01, -5.5057e-01,  1.3349e-01,  ..., -4.0768e-01,
           -1.3365e-01,  2.3631e-01],
          [-6.0551e-01,  5.2899e-03,  2.7282e-01,  ...,  4.0820e-01,
           -2.5521e-01, -1.2943e+00],
          [ 4.2050e-01,  1.2412e-01,  2.1755e-01,  ...,  1.2297e+00,
           -6.6092e-03, -2.1902e-01]],

         [[-7.1603e-01,  2.1500e-01, -1.6426e-01,  ..., -5.2377e-01,
            4.3221e-01, -1.6701e-01],
          [-3.8626e-02, -1.5478e+00,  1.9000e-01,  ..., -5.5221e-01,
           -6.0916e-01, -1.3324e+00],
          [-3.0654e-01,  2.3717e-01,  3.7409e-01,  ..., -3.4574e-02,
           -5.2827e-01,  8.7119e-01],
          ...,
     

In [11]:
x.dtype

torch.float32

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor